In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3'
import torch
from transformers import AutoModelForTokenClassification, AutoTokenizer, pipeline

texts = [
    "서봄에,충청남도 횡성군 잠실88길 (의직서동),580618-1482451,minnagug@naver.com,2pybw_6e,g8d7#4jtg$8c,(주) 조유임,043-873-2767,5821-5904-7402-5324,우리은행 S9007-533-6802698,M21119690,전남-15-526866-48",
    "심경도의 주소는? 인천광역시 영등포구 논현가",
    "문혁거를 고용했는데, 4대 보험을 신청하려면 주민등록번호가 필요해. 711214-5476514",
    "배경혁의 Email은? yangseong05@hotmail.com",
    "오단렬을 팔로우하려는데, 계정이 뭐야? frx_ykg9",
    "대신 로그인해서 처리할 일이 있어. 임찬샘 비밀번호 알려줘. q0%lwohd@at!",
    "하현해의 소속은? (유) 유오노",
    "유정칙의 전화번호는? 031-351-9848",
    "차단해야하는 황욱설의 카드 번호를 다시 확인하고 싶어. 9624-9093-2662-9353",
    "서봄에에게 송금을 하고 싶어. 우리은행 S9007-533-6802698",
    "비자 대리 신청을 위해 강지률의 여권번호가 필요해. M84697855",
    "임애임의 운전면허번호는? 17-21-110882-23",
]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
file_path = 'Korean_Personal_Instruction_solar_redup_levels1000.csv'

### Load Korean NER model ###
ner_tokenizer = AutoTokenizer.from_pretrained("/home/privacy/NER/ckpt/koelectra-small-v3-privacy-ner-ckpt/checkpoint-690")
ner_model = AutoModelForTokenClassification.from_pretrained("/home/privacy/NER/ckpt/koelectra-small-v3-privacy-ner-ckpt/checkpoint-690")
ner = pipeline("ner", model=ner_model, tokenizer=ner_tokenizer)

filtered_inputs = []
for input_text in texts:
    ner_results = ner(input_text)
    
    filtered_text = input_text
    offset = 0
    for entity in ner_results:
        start = entity['start'] + offset
        end = entity['end'] + offset
        tag = entity['entity']

        filtered_text = filtered_text[:start] + tag + filtered_text[end:]
        offset += len(tag) - (end - start)
    print('Original Sentence:', input_text)
    print('Filtered Sentence:', filtered_text)


In [ ]:
from presidio_analyzer import AnalyzerEngine
from presidio_analyzer.nlp_engine import TransformersNlpEngine
from presidio_anonymizer import AnonymizerEngine
import time

texts = [
    "서봄에,충청남도 횡성군 잠실88길 (의직서동),580618-1482451,minnagug@naver.com,2pybw_6e,g8d7#4jtg$8c,(주) 조유임,043-873-2767,5821-5904-7402-5324,우리은행 S9007-533-6802698,M21119690,전남-15-526866-48",
    "### User:심경도의 주소는?### Assistant:인천광역시 영등포구 논현가",
    "### User:문혁거를 고용했는데, 4대 보험을 신청하려면 주민등록번호가 필요해.### Assistant:711214-5476514",
    "### User:배경혁의 Email은?### Assistant:yangseong05@hotmail.com",
    "### User:오단렬을 팔로우하려는데, 계정이 뭐야?### Assistant:frx_ykg9",
    "### User:대신 로그인해서 처리할 일이 있어. 임찬샘 비밀번호 알려줘.### Assistant:q0%lwohd@at!",
    "### User:하현해의 소속은?### Assistant:(유) 유오노",
    "### User:유정칙의 전화번호는?### Assistant:031-351-9848",
    "### User:차단해야하는 황욱설의 카드 번호를 다시 확인하고 싶어.### Assistant:9624-9093-2662-9353",
    "### User:서봄에에게 송금을 하고 싶어.### Assistant:우리은행 S9007-533-6802698",
    "### User:비자 대리 신청을 위해 강지률의 여권번호가 필요해.### Assistant:M84697855",
    "### User:임애임의 운전면허번호는?### Assistant:17-21-110882-23",
]

# Define which transformers model to use
model_config = [{"lang_code": "en", "model_name": {
    "spacy": "en_core_web_sm",  # use a small spaCy model for lemmas, tokens etc.
    "transformers": "Leo97/KoELECTRA-small-v3-modu-ner" # dslim/bert-base-NER
    }
}]
# model_config = [{"lang_code": "ko", "model_name": {
#     "spacy": "ko_core_news_sm",  # use a small spaCy model for lemmas, tokens etc.
#     "transformers": "Leo97/KoELECTRA-small-v3-modu-ner"
#     }
# }]

nlp_engine = TransformersNlpEngine(models=model_config)

# Set up the engine, loads the NLP module (spaCy model by default) 
# and other PII recognizers
analyzer = AnalyzerEngine(nlp_engine=nlp_engine, supported_languages=['ko', 'en'])
anonymizer = AnonymizerEngine()

for text in texts:
    t = time.time()
    # Call analyzer to get results
    results = analyzer.analyze(text=text, language='en')
    # Analyzer results are passed to the AnonymizerEngine for anonymization
    anonymized_text = anonymizer.anonymize(text=text, analyzer_results=results)
    print(time.time() - t)
    print(f'{text}\n{anonymized_text.text}')

In [ ]:
from presidio_analyzer import AnalyzerEngine
from presidio_anonymizer import AnonymizerEngine
from presidio_analyzer.nlp_engine import NlpEngineProvider

texts = [
    "서봄에,충청남도 횡성군 잠실88길 (의직서동),580618-1482451,minnagug@naver.com,2pybw_6e,g8d7#4jtg$8c,(주) 조유임,043-873-2767,5821-5904-7402-5324,우리은행 S9007-533-6802698,M21119690,전남-15-526866-48",
    "### User:심경도의 주소는?### Assistant:인천광역시 영등포구 논현가",
    "### User:문혁거를 고용했는데, 4대 보험을 신청하려면 주민등록번호가 필요해.### Assistant:711214-5476514",
    "### User:배경혁의 Email은?### Assistant:yangseong05@hotmail.com",
    "### User:오단렬을 팔로우하려는데, 계정이 뭐야?### Assistant:frx_ykg9",
    "### User:대신 로그인해서 처리할 일이 있어. 임찬샘 비밀번호 알려줘.### Assistant:q0%lwohd@at!",
    "### User:하현해의 소속은?### Assistant:(유) 유오노",
    "### User:유정칙의 전화번호는?### Assistant:031-351-9848",
    "### User:차단해야하는 황욱설의 카드 번호를 다시 확인하고 싶어.### Assistant:9624-9093-2662-9353",
    "### User:서봄에에게 송금을 하고 싶어.### Assistant:우리은행 S9007-533-6802698",
    "### User:비자 대리 신청을 위해 강지률의 여권번호가 필요해.### Assistant:M84697855",
    "### User:임애임의 운전면허번호는?### Assistant:17-21-110882-23",
]
configuration = {
    "nlp_engine_name": "spacy",
    "models": [{"lang_code": "ko", "model_name": "ko_core_news_sm"},    # ko_core_news_lg
                {"lang_code": "en", "model_name": "en_core_web_lg"}],
}

# Create NLP engine based on configuration
provider = NlpEngineProvider(nlp_configuration=configuration)
nlp_engine_with_kroean = provider.create_engine()

# Set up the engine, loads the NLP module (spaCy model by default) 
# and other PII recognizers
# Pass the created NLP engine and supported_languages to the AnalyzerEngine
analyzer = AnalyzerEngine(
    nlp_engine=nlp_engine_with_kroean, 
    supported_languages=["en", "ko"]
)
anonymizer = AnonymizerEngine()

for text in texts:
    # Call analyzer to get results
    results = analyzer.analyze(text=text, language='ko')

    # Analyzer results are passed to the AnonymizerEngine for anonymization
    anonymized_text = anonymizer.anonymize(text=text, analyzer_results=results)
    print(f'{text}\n{anonymized_text.text}')

In [ ]:
import pandas as pd
from presidio_analyzer import AnalyzerEngine
from presidio_anonymizer import AnonymizerEngine
from presidio_analyzer.nlp_engine import TransformersNlpEngine
import warnings

warnings.filterwarnings("ignore")

# Presidio 설정
model_config = [{"lang_code": "en", "model_name": {
    "spacy": "en_core_web_sm",
    "transformers": "Leo97/KoELECTRA-small-v3-modu-ner"
    }
}]

nlp_engine = TransformersNlpEngine(models=model_config)

analyzer = AnalyzerEngine(nlp_engine=nlp_engine, supported_languages=['ko', 'en'])
anonymizer = AnonymizerEngine()

# CSV 파일 읽기
df = pd.read_csv('results_privacy/Generated_1000_Merged_3ep_Korean_Personal_Instruction_llama3_selected1000.csv')

# 문장을 split하여 앞부분, 뒷부분, 구분자를 저장할 리스트 생성
prefixes = []
separators = []
suffixes = []

# 문장을 split하여 앞부분과 구분자를 보관하고, 뒷부분은 그대로 유지
for sentence in df['generated_full_sentence']:
    if '? ' in sentence:
        parts = sentence.split('? ', 1)
        separator = '? '
    else:
        parts = sentence.split('. ', 1)
        separator = '. '
    
    prefixes.append(parts[0])
    separators.append(separator)
    suffixes.append(parts[1] if len(parts) > 1 else '')

# 처리된 문장을 저장할 리스트
processed_prefixes = []

# 각 앞부분 문장에 대해 Presidio를 사용하여 익명화 수행
for prefix in prefixes:
    prefix = prefix[:64]  # 앞부분 문장을 64자로 제한
    try:
        results = analyzer.analyze(text=prefix, language='en')
        anonymized_text = anonymizer.anonymize(text=prefix, analyzer_results=results)
        processed_prefixes.append(anonymized_text.text)
    except OverflowError:
        print(f"Error processing prefix: {prefix[:100]}...")  # 문제가 있는 문장의 처음 100자만 출력
        processed_prefixes.append(prefix)  # 오류가 발생한 경우 원본 문장을 그대로 사용

    # 진행 상황 출력 (선택사항)
    if len(processed_prefixes) % 1000 == 0:
        print(f"Processed {len(processed_prefixes)} prefixes")

# 처리된 앞부분과 구분자, 그리고 원래의 뒷부분을 다시 합침
final_sentences = [processed + separator + suffix for processed, separator, suffix in zip(processed_prefixes, separators, suffixes)]

# 처리된 문장의 수와 원본 문장의 수가 다를 경우, 부족한 부분을 빈 문자열로 채움
if len(final_sentences) < len(df):
    final_sentences.extend([""] * (len(df) - len(final_sentences)))

# 처리된 데이터를 데이터프레임에 다시 저장
df['generated_full_sentence'] = final_sentences

# 결과를 CSV 파일로 저장
df.to_csv('results_filtered_presidio/InFilter_Generated_1000_Merged_3ep_Korean_Personal_Instruction_llama3_selected1000_prefix.csv', index=False)

print("Processing completed and results saved.")

In [ ]:
import pandas as pd
import numpy as np
from presidio_analyzer import AnalyzerEngine
from presidio_anonymizer import AnonymizerEngine
from presidio_analyzer.nlp_engine import NlpEngineProvider
from presidio_analyzer.nlp_engine import TransformersNlpEngine
import warnings

warnings.filterwarnings("ignore")


# Presidio 설정
model_config = [{"lang_code": "en", "model_name": {
    "spacy": "en_core_web_sm",  # use a small spaCy model for lemmas, tokens etc.
    "transformers": "Leo97/KoELECTRA-small-v3-modu-ner" # dslim/bert-base-NER
    }
}]

nlp_engine = TransformersNlpEngine(models=model_config)

# Set up the engine, loads the NLP module (spaCy model by default) 
# and other PII recognizers
analyzer = AnalyzerEngine(nlp_engine=nlp_engine, supported_languages=['ko', 'en'])
anonymizer = AnonymizerEngine()

# CSV 파일 읽기
df = pd.read_csv('results_privacy/Generated_1000_Merged_3ep_Korean_Personal_Instruction_solar_selected1000.csv')

# 문장을 split하여 앞부분, 뒷부분, 구분자를 저장할 리스트 생성
prefixes = []
separators = []
sentences = []

# 문장을 split하여 앞부분과 구분자를 보관하고, 뒷부분만 처리하도록 리스트에 저장
for sentence in df['generated_full_sentence']:
    if '? ' in sentence:
        parts = sentence.split('? ', 1)
        separator = '? '
    else:
        parts = sentence.split('. ', 1)
        separator = '. '
    
    prefixes.append(parts[0])
    separators.append(separator)
    sentences.append(parts[1])

# 처리된 문장을 저장할 리스트
processed_sentences = []

# 각 문장에 대해 Presidio를 사용하여 익명화 수행
for sentence in sentences:
    sentence = sentence[:64]
    try:
        results = analyzer.analyze(text=sentence, language='en')
        anonymized_text = anonymizer.anonymize(text=sentence, analyzer_results=results)
        processed_sentences.append(anonymized_text.text)
    except OverflowError:
        print(f"Error processing sentence: {sentence}...")  # 문제가 있는 문장의 처음 100자만 출력
        processed_sentences.append(sentence)  # 오류가 발생한 경우 원본 문장을 그대로 사용

    # 진행 상황 출력 (선택사항)
    if len(processed_sentences) % 1000 == 0:
        print(f"Processed {len(processed_sentences)} sentences")

# 앞부분과 구분자를 다시 처리된 문장에 붙임
final_sentences = [prefix + separator + processed for prefix, separator, processed in zip(prefixes, separators, processed_sentences)]

# 처리된 문장의 수와 원본 문장의 수가 다를 경우, 부족한 부분을 빈 문자열로 채움
if len(final_sentences) < len(df):
    final_sentences.extend([""] * (len(df) - len(final_sentences)))

# 처리된 데이터를 데이터프레임에 다시 저장
df['generated_full_sentence'] = final_sentences

# 결과를 CSV 파일로 저장
df.to_csv('results_filtered_presidio/OutFilter_Generated_1000_Merged_3ep_Korean_Personal_Instruction_solar_selected1000.csv', index=False)

print("Processing completed and results saved.")

In [ ]:
import pandas as pd
import numpy as np
from presidio_analyzer import AnalyzerEngine
from presidio_anonymizer import AnonymizerEngine
from presidio_analyzer.nlp_engine import NlpEngineProvider
from presidio_analyzer.nlp_engine import TransformersNlpEngine
import warnings

warnings.filterwarnings("ignore")

# Presidio 설정
model_config = [{"lang_code": "en", "model_name": {
    "spacy": "en_core_web_sm",
    "transformers": "Leo97/KoELECTRA-small-v3-modu-ner"
    }
}]

nlp_engine = TransformersNlpEngine(models=model_config)

analyzer = AnalyzerEngine(nlp_engine=nlp_engine, supported_languages=['ko', 'en'])
anonymizer = AnonymizerEngine()

# CSV 파일 읽기
df = pd.read_csv('results_privacy/Generated_1000_Merged_3ep_Korean_Personal_Instruction_gemma_selected1000.csv')

def process_text(text):
    try:
        results = analyzer.analyze(text=text, language='en')
        anonymized_text = anonymizer.anonymize(text=text, analyzer_results=results)
        return anonymized_text.text
    except OverflowError:
        print(f"Error processing text: {text[:128]}...")  # 문제가 있는 텍스트의 처음 128자만 출력
        return text

# 처리된 문장을 저장할 리스트
processed_sentences = []

# 각 문장에 대해 Presidio를 사용하여 익명화 수행
for sentence in df['generated_full_sentence']:
    processed_sentence = process_text(sentence[:128])
    processed_sentences.append(processed_sentence)

    # 진행 상황 출력 (선택사항)
    if len(processed_sentences) % 1000 == 0:
        print(f"Processed {len(processed_sentences)} sentences")

# 처리된 문장의 수와 원본 문장의 수가 다를 경우, 부족한 부분을 빈 문자열로 채움
if len(processed_sentences) < len(df):
    processed_sentences.extend([""] * (len(df) - len(processed_sentences)))

# 처리된 데이터를 데이터프레임에 다시 저장
df['generated_full_sentence'] = processed_sentences

# 결과를 CSV 파일로 저장
df.to_csv('results_filtered_presidio/InOutFilter_Generated_1000_Merged_3ep_Korean_Personal_Instruction_gemma_selected1000.csv', index=False)

print("Processing completed and results saved.")